In [1]:
import temporal_graph
import matplotlib.pyplot as plt


In [2]:
graph = temporal_graph.TemporalGraph()
graph.clean_nodes(True, True, False)


creating time intervall ...
the time intervall of the data goes
from: 2022-02-07 23:31:00 
to: 2022-10-15 21:14:00
we have the following time intervalls[(datetime.datetime(2022, 2, 7, 23, 31), datetime.datetime(2022, 2, 14, 23, 31)), (datetime.datetime(2022, 2, 14, 23, 31), datetime.datetime(2022, 2, 21, 23, 31)), (datetime.datetime(2022, 2, 21, 23, 31), datetime.datetime(2022, 2, 28, 23, 31)), (datetime.datetime(2022, 2, 28, 23, 31), datetime.datetime(2022, 3, 7, 23, 31)), (datetime.datetime(2022, 3, 7, 23, 31), datetime.datetime(2022, 3, 14, 23, 31)), (datetime.datetime(2022, 3, 14, 23, 31), datetime.datetime(2022, 3, 21, 23, 31)), (datetime.datetime(2022, 3, 21, 23, 31), datetime.datetime(2022, 3, 28, 23, 31)), (datetime.datetime(2022, 3, 28, 23, 31), datetime.datetime(2022, 4, 4, 23, 31)), (datetime.datetime(2022, 4, 4, 23, 31), datetime.datetime(2022, 4, 11, 23, 31)), (datetime.datetime(2022, 4, 11, 23, 31), datetime.datetime(2022, 4, 18, 23, 31)), (datetime.datetime(2022, 4, 18, 

100%|██████████| 454458/454458 [00:01<00:00, 344162.30it/s]


done.
assigning hate speech labels to truths ...


100%|██████████| 454458/454458 [00:01<00:00, 305513.63it/s]


assigning truth to truth relations ...


100%|██████████| 454458/454458 [00:00<00:00, 696077.20it/s] 


cleaning nodes ...
nodes cleaned ...


In [7]:
import random

# value = random.choice(list(graph.nodes_dict.values()))
value = graph.nodes_dict["3831"]
value.node_info()


time_intervalls: <temporal_graph.TimeIntervall object at 0x0000021BB3965010>
truths_in_time_intervall: [[], [], ['827621', '827610', '827634', '827675'], ['827616', '827654', '827679', '827688', '827692'], ['827564', '827578', '827608', '827612', '827613', '827618', '827619', '827635', '827636', '827637', '827639', '827605', '827611', '827620', '827623', '827624', '827617', '827625', '827627', '827628', '827629', '827630', '827632', '827633', '827640', '827642', '827647', '827649', '827651', '827657', '827659', '827661', '827663', '827665', '827669', '827671', '827673', '827677', '827681', '827683', '827631', '827644', '827667'], ['827607', '827579', '827582', '827585', '827593', '827594', '827586', '827587', '827588', '827590', '827595', '827596', '827598', '827599', '827600', '827601', '827603', '827604', '827606'], ['827580', '827581', '827583', '827577'], ['827568', '827572', '827573', '827574', '827576'], ['827565', '827566', '827567', '827569'], ['827560', '827561', '827562'], ['

In [6]:
graph.find_truth("807621")

timestamp: 2022-03-26 00:00:00
id: 807621
text: Joe Biden wants this video removed from the internet. I say we up the game and share it all over
like_count: 0
retruth_count: 0
reply_count: 0
emotion: neutral
emotion_logits: [1.7952251434326172, 0.087712250649929, -1.6980016231536863, -2.171093463897705, 3.135545253753662, 0.1451661437749862, -0.8349509835243225]
hate_speech_label: 0
hate_speech_prob: 0.015105183
sentiment_id: 0
sentiment_conf: 0.81784797
sentiment: negative
statement_flag: 1
statement_probability: 0.519844114780426
probabilites: {'TRUE': 0.02, 'FALSE': 0.08, 'NO_STATEMENT': 0.9}


In [4]:
for truth_id in graph.nodes_dict["20054"].truths:
    print(type(truth_id))


<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class

In [5]:
graph.create_pyvis_representation()


creating image of network...


100%|██████████| 17123/17123 [00:07<00:00, 2319.84it/s]


NetworkXNotImplemented: not implemented for directed type

In [ ]:
following_edges_per_node = []
follower_edges_per_node = []
messages_per_node = []

counter_follower = 0
counter_following = 0
counter_message = 0

zero_counter_following = 0
zero_counter_follower = 0
zero_counter_message = 0

for key in graph.nodes_dict.keys():
    if len(graph.nodes_dict[key].following_edges) != 0:
        following_edges_per_node.append(len(graph.nodes_dict[key].following_edges))
        counter_following += 1
    else:
        zero_counter_following += 1

    if len(graph.nodes_dict[key].truths) != 0:
        messages_per_node.append(len(graph.nodes_dict[key].truths))
        counter_message += 1
    else:
        zero_counter_message += 1

    if len(graph.nodes_dict[key].follower_edges) != 0:
        follower_edges_per_node.append(len(graph.nodes_dict[key].follower_edges))
        counter_follower += 1
    else:
        zero_counter_follower += 1

print(f"nodes with no known followings: {zero_counter_following} nodes with followings:{counter_following}")
print(f"nodes with no known messages: {zero_counter_message} with known messages:{counter_message}")
print(f"nodes with no known followers: {zero_counter_follower} nodes with followers:{counter_follower}")


In [ ]:
import numpy as np

print(np.median(following_edges_per_node))
plt.hist(following_edges_per_node, bins=3000)
plt.xscale("log")
plt.yscale("log")


In [ ]:
plt.hist(follower_edges_per_node, bins=3000)
plt.xscale("log")
plt.yscale("log")


In [ ]:
plt.hist(messages_per_node, bins=3000)
plt.xscale("log")
plt.yscale("log")
